<a href="https://colab.research.google.com/github/dtht2d/deep-learning/blob/main/codes/face-recognition/practical_face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%autosave 30

Autosaving every 30 seconds


# **Fine-tuning VGG19 for Face Recognition: Analysis of Genuine and Impostor Scores, ROC Curve, and Threshold Optimization**

# **Objective**
Fine-tune a VGG19 with 70% of the data[1]
(7 images of each identity) and then use fc7 output as features (do away with classification head but after training-see MATLAB help for “activations” to see how you can get the outputs of an intermediate layer). Using the remaining 3 images per identity (30% validation, that would be images 8, 9, and 10 for each subject), and cosine similarity between enrollment (first validation image, #8) and verifications (second and third images from validation, images #9 and #10), create genuine and impostor score sets and plot their distributions (histograms) and ROC of these two classes (include AUC) for training and testing subsets [2]

Cosine similarity between vesctors X and Y is given by 
$$
cos(\theta)==\frac{\sum{x_i*y_i}}{\sqrt{\sum{x^2_i}}*\sqrt{\sum{y^2_i}}}
$$

**Other tasks**:

- Use ROC curves or score sets, find the threshold for ≈ 1% FAR on the training set, report the threshold and the corresponding training GAR
- Apply that decision threshold to the validation score. Report on the resultig FAR and GAR 
---
[1] AT&T face recognition dataset from https://www.kaggle.com/kasikrit/att-database-of-faces

[2]To generate all permutations for genuine and imposter pairs, say for the test subset, assume the first verification2
feature vector for subject 1 (enrollment reference) is f1_8. Compare it (using cosine similarity) with the remaining 2
feature vectors from subject 1: f1_9, f1_10, and record the cosine similarity scores. Now compare f1_8 with the last 2
verification features of subject 2: f2_9, f2_10, and record the cosine similarity scores. Continue until you compare
f1_8 with the last 2 verification features of the last subject 40: f40_9, f40_10, and record the cosine similarity scores.
Carry out a similar process to generate training genuine and impostor scores.



## **Purpose** 

### Fine-tuning VGG19 for Face Recognition: Analysis of Genuine and Impostor Scores using Cosine Similarity and ROC Curve

1. Feature Extraction: By fine-tuning the VGG19 model and extracting features from the fc7 layer, we obtain a lower-dimensional representation of the images that captures important patterns and discriminative information. This allows us to leverage the pre-trained model's knowledge from a large dataset (ImageNet) and adapt it to our specific task.

2. Verification and Identification: By comparing the enrollment image (a reference image) with the verification images using cosine similarity, we can quantify the similarity between these images. This process is commonly used in face recognition or biometric identification systems to determine whether two images belong to the same person (genuine scores) or different people (impostor scores).

3. Performance Evaluation: Analyzing the distribution of genuine and impostor scores using histograms provides insights into the separability of the features extracted by the model. A well-performing model should have distinct distributions for genuine and impostor scores, indicating that genuine pairs have higher similarity scores than impostor pairs. This evaluation helps understand the model's ability to discriminate between genuine and impostor samples.

    ROC Curve and AUC: The ROC curve is a visual representation of the performance of a binary classification system across different thresholds. It shows the trade-off between the true positive rate (sensitivity) and the false positive rate (1 - specificity). The AUC (Area Under the Curve) summarizes the overall performance of the classifier, where a higher AUC value indicates better discrimination between genuine and impostor scores. The ROC curve and AUC provide a comprehensive evaluation of the model's ability to separate the two classes.

By following these steps and evaluating the model's performance using metrics like ROC and AUC, you can assess the effectiveness of the fine-tuned VGG19 model for your specific task, understand the separability of the features, and determine its capability to distinguish between genuine and impostor pairs. This analysis is particularly valuable in applications such as biometric recognition or verification systems, where accurate discrimination between individuals is essential.

### Finding the threshold 

The purpose of finding the threshold for approximately 1% False Acceptance Rate (FAR) on the training set and applying it to the validation scores is to assess the performance of the face recognition system at a specific operating point.
- False Acceptance Rate (FAR): FAR represents the probability of the system incorrectly accepting an impostor (mismatched) pair as a genuine match. A lower FAR indicates a more secure system, as it means fewer impostor pairs are falsely accepted.

- Genuine Acceptance Rate (GAR): GAR represents the probability of the system correctly accepting a genuine (matched) pair. A higher GAR indicates a higher accuracy in recognizing genuine pairs.

By finding the threshold for approximately 1% FAR on the training set, we determine the threshold score that achieves a desirable balance between system security and recognition accuracy. This threshold separates the genuine and impostor scores, allowing us to classify them as matches or non-matches.

Applying this threshold to the validation scores allows you to evaluate the system's performance on **unseen data**. We can measure the resulting False Acceptance Rate (FAR) and Genuine Acceptance Rate (GAR) on the validation set, which indicates how well the system performs in real-world scenarios.

By reporting the threshold and the corresponding training GAR, we provide insights into the system's performance at the specified operating point. Similarly, reporting the resulting FAR and GAR on the validation set helps assess the generalization and effectiveness of the system on new data.

Overall, this analysis allows you to fine-tune the system's threshold to achieve a desired balance between security (FAR) and recognition accuracy (GAR) and evaluate its performance on both training and validation data.

## **II. Methods**


- Fine-tune a VGG19 with 70% of the data1
(7 images of each identity) and then use fc7 output as features (do away with classification head
but after training-see MATLAB help for “activations” to see how you can get the outputs of an
intermediate layer). Using the remaining 3 images per identity (30% validation, that would be
images 8, 9, and 10 for each subject), and cosine similarity between enrollment (first validation
image, #8) and verifications (second and third images from validation, images #9 and #10),
create genuine and impostor score sets and plot their distributions (histograms) and ROC of
these two classes (include AUC) for training and testing subsets .2

Note: Cosine similarity between vectors X and Y is given by
Bonus (up to 15 points, if done correctly): using your ROC curves or score sets, find the
threshold for ~1% FAR on the training set, and report the threshold and the corresponding
training GAR (5 points). 2-Apply that decision threshold to the validation scores. Report on the
resulting FAR and GAR (10 points).
Deliverable: Please submit your original individual work in form of a slide deck (describing your
results (as requested above) and your observations, plus all your MATLAB (or Python) code and
NN models (but not the dataset). Submit everything electronically here via CANVAS ONLY by
midnight, Friday, April 28.
Please download the AT&T face recognition dataset from https://www.kaggle.com/kasikrit/att-database-of-faces (401
subjects with 10 different mugshots for each). Follow MATLAB’s transfer learning example https://
www.mathworks.com/help/deeplearning/examples/transfer-learning-using-alexnet.html but use VGG19. Other similar
architectures such as ResNet or SqueezeNet are also acceptable. You may carry out the project in Python, if you
prefer.
To generate all permutations for genuine and imposter pairs, say for the test subset, assume the first verification2
feature vector for subject 1 (enrollment reference) is f1_8. Compare it (using cosine similarity) with the remaining 2
feature vectors from subject 1: f1_9, f1_10, and record the cosine similarity scores. Now compare f1_8 with the last 2
verification features of subject 2: f2_9, f2_10, and record the cosine similarity scores. Continue until you compare
f1_8 with the last 2 verification features of the last subject 40: f40_9, f40_10, and record the cosine similarity scores.
Carry out a similar process to generate training genuine and impostor scores.
Annotations
